 # Test the tuned Model

 In the last notebook, we have seen the steps to fine tune AlexNet model with MNIST dataset.
 In this notebook, we will load the tuned model shared by NICD and then will test it with MNIST data.

 In the import section below, we have used two new import which are torch.nn and matplotlib.pyplot.
 In which torch.nn provide us many more classes and modules to implement and train the neural network and
 matplotlib.pyplot is a collection of functions that make matplotlib work like MATLAB through which we can create figures.

In [ ]:
import torch
from torchvision import models, transforms, datasets
import torch.nn as nn
import matplotlib.pyplot as plt


 The Transforms module used in the below snippet helped us in transforming the greyscaled MNIST data into a tensor.


In [ ]:
transform = transforms.Compose([
 transforms.Resize(224),
 transforms.ToTensor(),
 # MNIST is greyscale so we stack the image to create 3 channels
 transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
 transforms.Normalize(
 mean=[0.485, 0.456, 0.406],
 std=[0.229, 0.224, 0.225]),
 ])


 Torchvision provides many built-in datasets in the torchvision.datasets module.
 we have downloaded the MNIST datasets here and stored it in the directory named as Data.
 Data loader used here combines a dataset and a sampler, and provides an iterable over the given dataset.

In [ ]:
mnist_test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
MNIST_test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=1, shuffle=True)


 In the below snippet, we have created a alexnet model without its pretrained feature.
 If we execute the model, we can see the model architecture with out_feature =1000 in its last layer.

In [ ]:
model = models.alexnet()
model

 # Redefining the layers
 The task is to classify a given image of a handwritten digit into one of 10 classes representing integer values from 0 to 9, inclusively.
 So we need to replace the last layer with 10 neurons.
 nn. Linear(n,m) is a module that creates single layer feed forward network with n inputs and m output.
 Through which we can update the output of our last layer.

In [ ]:
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs,10)


 If we check the model architecture, we can see the last layer out_features was updated to 10.

In [ ]:
model


 Now, we have our expected model architecture for MNIST data pretection.
 But it needs to be trained to predict MNIST data. Instead of training the model with MNIST dataset, we directly loaded the finetuned alexnet model head shared by NICD.
 Training a model in a CPU machine will take a long time.
 So with the help of NICD, we finttuned our pretrained alexnet model with MNIST dataset in NICD's GPU machine which took approximately 40 to 50 mins for 50 epochs.

 To load this finetuned model, we need to place the model file (mnist_head_alexnet.pth) in the same directory of our source code.
 Then using torch.load() we can deserialize python's object files to memory.
 Finally, using load_state_dict, we can loads a model’s parameter dictionary using a deserialized state_dict.

 In PyTorch, the learnable parameters (i.e. weights and biases) of an torch.nn.Module model are contained in the model’s parameters (accessed with model.parameters()).
 A state_dict is simply a Python dictionary object that maps each layer to its parameter tensor.
 Note that only layers with learnable parameters (convolutional layers, linear layers, etc.) and registered buffers have entries in the model’s state_dict.

In [ ]:
model.load_state_dict(torch.load('mnist_head_alexnet.pth',map_location=torch.device('cpu')))
model.eval()


 Since, we are going to test the model directly we are using eval mode in the above snippet.
 Next, we need to pass our MNIST data into our model and predict the output.
 So, using the below code snippet we iterated and picked a single data from our dataset.

In [ ]:
sample_image = iter(MNIST_test_loader).next()[0]
sample_image

 Our sample image is in tensor format.
 To visualise the selected data, we used matplotlib.pyplot to create the figure of the image.

In [ ]:
pil_iamge = transforms.ToPILImage()(sample_image.squeeze(0))
plt.imshow(pil_iamge, cmap='gray')


 Finally, we just passed our sample test data into our model and
 model precits the probablities of each tensor[0-9].

In [ ]:
pred =model(sample_image)
pred

In [ ]:
torch.argmax(pred)


 Finally, using torch.argmax tensor with highest probabilties is returned and that is our expected output.